# Description
In this notebook methods to find the ags for factories.

# OpenToDos:
- What to do with AGS = DG00000?

In [ ]:
import pandas as pd
import numpy as np

# Load data

In [ ]:
def add_zeros(ags):
    ags = str(ags)
    length_ags = len(ags)
    if length_ags < 8:
        ags = "0" * (8 - length_ags) + ags
    return ags


df_ags = pd.read_csv("../data/public/ags/master.csv")

df_anlagen = pd.read_excel(
    "Anlagenliste_2018_DEHST.xlsx",
    sheet_name="Anlagen_Liste",
    decimal=",",
    thousands=".",
    converters={"Bundesland": str, "AGS_manual": str, "VET 2018 [t CO2 Äq]": float},
)

df_anlagen.rename(
    columns={
        "Standort der Anlage": "Ort",
        "AGS": "AGS_xls",
        "VET 2018 [t CO2 Äq]": "VET2018_t",
        "Haupttätigkeit nach TEHG": "TEHG_Nr",
        "Bezeichnung Haupttätigkeit nach TEHG": "TEHG_name",
    },
    inplace=True,
)
# df_anlagen["AGS_manual"] = df_anlagen["AGS_manual"].apply(add_zeros)
df_anlagen.drop(
    columns=[
        "Prüfsumme",
    ],
    inplace=True,
)
df_anlagen.dropna(subset=["NACE WZ2008"], inplace=True)


# Data Processing
## Add industrie cateogires to facilities
Look at [Sharepoint](https://germanzero.sharepoint.com/:w:/r/Files/200_Campaigning_Mobilisierung/10_Klimaentscheide/06_LocalZero/03_Generator/20_KStP-Generator_v2/30_Industrie/70_Kommunenfeine_Bilanz/22-07-14_Methodischer%20Ansatz%20kommunenfeine%20Industrie.docx?d=wd6874aa0f4f944e19c6a29dda796c406&csf=1&web=1&e=SnSbBP) for further information about Nace

In [ ]:
# only usable for 2018 VET as not all NACE identifiers are added (only the ones present in the excel sheet)
mapper_nace_wz = {
    "miner_cement": ["23.51"],
    "miner_chalk": ["23.52", "23.62"],
    "miner_glas": ["23.1", "23.99"],
    "miner_ceram": ["23.2", "23.3", "23.4"],
    "chem_basic": ["20.11", "20.12", "20.13", "20.14", "20.16"],
    "chem_ammonia": ["20.15"],
    "chem_other": ["20.2", "20.3", "20.4", "20.5", "20.6", "21."],
    "metal_steel": ["24.1"], # primary and secondary is sorted by cities later
    "metal_nonfe": [
        "24.4", "24.5"
    ],
    "other_paper": ["17."],
    "other_food": ["10.8", "10.6","10.5", "10.4", "10.3", "11.05", "12.00"],
    "other_further": ["8.", "22.", "24.2", "24.3", "25.", "28.1", "29.", "30."],
}


In [ ]:
df_anlagen["i_category"] = "No category"
for key, item in mapper_nace_wz.items():
    if len(item) > 0:
        for name in item:
            df_anlagen.loc[df_anlagen["NACE WZ2008"].str.contains(name), "i_category"] = key

## Select "Anlagen" from industry

In [ ]:
df_anlagen.groupby("i_category")["VET2018_t"].sum().sort_values(ascending=False)

In [ ]:
df_anlagen = df_anlagen[df_anlagen["i_category"] != "No category"]

In [ ]:
print(f"{len(df_anlagen)} selected")

## Add state information to df_ags

In [ ]:
ags_to_state = {
    "01" : "SH",
    "02" : "HH",
    "03" : "NI",
    "04" : "HB",
    "05" : "NW",
    "06" : "HE",
    "07" : "RP",
    "08" : "BW",
    "09" : "BY",
    "10" : "SL",
    "11" : "BE",
    "12" : "BB",
    "13" : "MV",
    "14" : "SN",
    "15" : "ST",
    "16" : "TH"
}

In [ ]:
df_ags["ags_state_digits"] = [x[:2] for x in df_ags["ags"]]
df_ags["state"] = df_ags["ags_state_digits"].map(ags_to_state)

## Remove not necessary parts of the Names in AGS

In [ ]:
# remove " , Stadt"
df_ags["name"] = df_ags["description"].str.split(",").str[0]
# df_ags.to_csv("temp_python_export_ags_list.csv", index=False)

## Select duplicates in AGS List (column: "description")

In [ ]:
df_ags_duplicates = df_ags[df_ags.duplicated(subset="name", keep=False)]
df_ags_duplicates = df_ags_duplicates.sort_values("name")
df_ags_no_duplicates = df_ags.drop_duplicates(subset="name", keep=False)

In [ ]:
n = 0
mask_duplicates_state_unique = [False] * len(df_ags_duplicates)
while True:
    row = df_ags_duplicates.iloc[n]
    ort_count = df_ags_duplicates[df_ags_duplicates["name"] == row["name"]].shape[0]

    if ort_count ==2:
        if row["state"] != df_ags_duplicates.iloc[n + 1]["state"]:
            mask_duplicates_state_unique[n] = True
            mask_duplicates_state_unique[n + 1] = True

    elif ort_count ==3:
        if (
            (row["state"] != df_ags_duplicates.iloc[n + 1]["state"])
            & (
                row["state"]
                != df_ags_duplicates.loc[:, "state"].iloc[n + 2]
            )
            & (
                df_ags_duplicates.loc[:, "state"].iloc[n + 1]
                != df_ags_duplicates.loc[:, "state"].iloc[n + 2]
            )
        ):
            mask_duplicates_state_unique[n] = True
            mask_duplicates_state_unique[n + 1] = True
            mask_duplicates_state_unique[n + 2] = True

    n += ort_count
    if n >= len(df_ags_duplicates):
        break

In [ ]:
df_ags_duplicates_state_unique = df_ags_duplicates[mask_duplicates_state_unique]

# Find AGS for "Ort"

In [ ]:
df_anlagen

In [ ]:
for index, row in df_anlagen.iterrows():
    state_anlage = row.loc["Bundesland"]
    location_anlage = row.loc["Ort"]
    if ~df_anlagen.loc[:,"AGS_manual"].isna().loc[index]:
        df_anlagen.loc[index, "AGS"] = df_anlagen.loc[index, "AGS_manual"]
        df_anlagen.loc[index, "match_type"] = "manual_entry"
    else:
        temp_ags_match = df_ags_no_duplicates.loc[(df_ags_no_duplicates["name"] == location_anlage)]
        # select entries with only one match
        if temp_ags_match.shape[0] == 1:
            if temp_ags_match["state"].values[0] == state_anlage:
                df_anlagen.loc[index, "AGS"] = temp_ags_match["ags"].values[0]
                df_anlagen.loc[index, "match_type"] = "direct"

        temp_ags_match_dup = df_ags_duplicates_state_unique.loc[(df_ags_duplicates_state_unique["name"] == location_anlage)]
        if temp_ags_match_dup.shape[0] == 1:
            raise ValueError("Duplication selection wrong. Single entry found in AGS list")
        elif temp_ags_match_dup.shape[0] > 1:
            for index_match, match in temp_ags_match_dup.iterrows():
                if match["state"] == state_anlage:
                    df_anlagen.loc[index, "AGS"] = match["ags"]
                    df_anlagen.loc[index, "match_type"] = "dublicate_match_state_unique"

In [ ]:

for index, row in df_anlagen[df_anlagen["match_type"].isna()].iterrows():
    state_anlage = row.loc["Bundesland"]
    location_anlage = row.loc["Ort"]
    temp_ags_match = df_ags_no_duplicates[df_ags_no_duplicates["name"].str.startswith(location_anlage) == True]
    if temp_ags_match.shape[0] == 1:
        if temp_ags_match["state"].values[0] == state_anlage:
            df_anlagen.loc[index, "AGS"] = temp_ags_match["ags"].values[0]
            df_anlagen.loc[index, "match_type"] = "starts_with_only_one_match"

In [ ]:
df_anlagen["match_type"].value_counts(dropna=False)

In [ ]:
df_anlagen[pd.isna(df_anlagen["match_type"])]

In [ ]:
df_anlagen.to_csv("export_anlagen_with_ags.csv")

In [ ]:
df_anlagen[pd.isna(df_anlagen["match_type"])].to_csv("anlagen_nan_ags.csv")

In [ ]:
temp =df_anlagen[df_anlagen["match_type"].isna()]

In [ ]:
temp.groupby("i_category")["VET2018_t"].sum()

In [ ]:
df_anlagen.groupby("i_category")["VET2018_t"].sum()

## Find steel plants and categroize to primal and secondary

TODO

In [ ]:
df_anlagen[df_anlagen["i_category"] == "metal_steel"]